In [1]:
import pandas as pd
import numpy as np

import spacy
from spacy.tokenizer import Tokenizer

from sklearn.externals import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

nlp = spacy.load("en_core_web_lg")

In [25]:
data1 = pd.read_csv('../merged.csv', na_values='None')
# data = data.set_index('name')
print(data.shape)
data1.head()

(2349, 11)


,id,name,race,flavors,positive,negative,medical,Rating,Description
0,1,Afpak,hybrid,"['Earthy', 'Chemical', 'Pine', 'Spicy/Herbal']","['Relaxed', 'Hungry', 'Happy', 'Sleepy', 'Crea...",['Dizzy'],"['Depression', 'Insomnia', 'Pain', 'Stress', '...",4.2,"Afpak, named for its direct Afghani and Pakist..."
1,2,African,sativa,"['Spicy/Herbal', 'Pungent', 'Earthy', 'Pepper']","['Euphoric', 'Happy', 'Creative', 'Energetic',...",['Dry Mouth'],"['Depression', 'Pain', 'Stress', 'Lack of Appe...",3.9,African refers to the indigenous varieties of ...
2,3,Afternoon Delight,hybrid,"['Pepper', 'Flowery', 'Pine', 'Pungent', 'Citr...","['Relaxed', 'Hungry', 'Euphoric', 'Uplifted', ...","['Dizzy', 'Dry Mouth', 'Paranoid']","['Depression', 'Insomnia', 'Pain', 'Stress', '...",4.8,"Afternoon Delight, created by Colorado Seed In..."
3,4,Afwreck,hybrid,"['Pine', 'Earthy', 'Flowery', 'Pungent']","['Relaxed', 'Happy', 'Creative', 'Uplifted', '...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Pain', 'Stress', 'Headache', 'Fatigue', 'Hea...",4.2,Afwreck is a hybrid cross of Afghani and Train...
4,5,Agent Orange,hybrid,"['Citrus', 'Orange', 'Sweet', 'Earthy']","['Relaxed', 'Euphoric', 'Happy', 'Energetic', ...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Depression', 'Pain', 'Stress', 'Nausea', 'He...",4.2,Don’t let the name scare you! The only herbici...


In [33]:
data1.medical.str.contains("['Depression', 'Stress', 'Fatigue']").sum()

1916

In [35]:
data.Medical.str.contains("Depression, Stress, Fatigue").sum()

931

In [17]:
data = pd.read_csv('../DS_strain_data.csv', na_values='None')
# data = data.set_index('name')
print(data.shape)
data.head()

(2349, 11)


,Unnamed: 0,Id,Strain,Race,Flavors,Positive,Negative,Medical,Rating,Description,everything
0,0,0,100 Og,hybrid,"Earthy, Sweet, Citrus","Creative, Energetic, Tingly, Euphoric, Relaxed","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.0,$100 OG is a 50/50 hybrid strain that packs a ...,"Earthy, Sweet, Citrus,Creative, Energetic, Tin..."
1,1,1,1024,sativa,"Spicy/Herbal, Sage, Woody","Uplifted, Happy, Relaxed, Energetic, Creative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,1024 is a sativa-dominant hybrid bred in Spain...,"Spicy/Herbal, Sage, Woody,Uplifted, Happy, Rel..."
2,2,2,13 Dawgs,hybrid,"Apricot, Citrus, Grapefruit","Tingly, Creative, Hungry, Relaxed, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Apricot, Citrus, Grapefruit,Tingly, Creative, ..."
3,3,3,24K Gold,hybrid,"Citrus, Earthy, Orange","Happy, Relaxed, Euphoric, Uplifted, Talkative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...","Citrus, Earthy, Orange,Happy, Relaxed, Euphori..."
4,4,4,3 Bears Og,indica,"Sweet, Pungent, Earthy","Relaxed, Happy, Sleepy, Creative, Euphoric","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,3 Bears OG by Mephisto Genetics is an autoflow...,"Sweet, Pungent, Earthy,Relaxed, Happy, Sleepy,..."


In [18]:
data.isnull().sum()

Unnamed: 0      0
Id              0
Strain          0
Race            0
Flavors        58
Positive       31
Negative        0
Medical         0
Rating          0
Description     5
everything      0
dtype: int64

In [19]:
tokenizer = Tokenizer(nlp.vocab)

In [20]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [24]:
data.head(100)

,Unnamed: 0,Id,Strain,Race,Flavors,Positive,Negative,Medical,Rating,Description,everything
0,0,0,100 Og,hybrid,"Earthy, Sweet, Citrus","Creative, Energetic, Tingly, Euphoric, Relaxed","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.0,$100 OG is a 50/50 hybrid strain that packs a ...,"Earthy, Sweet, Citrus,Creative, Energetic, Tin..."
1,1,1,1024,sativa,"Spicy/Herbal, Sage, Woody","Uplifted, Happy, Relaxed, Energetic, Creative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,1024 is a sativa-dominant hybrid bred in Spain...,"Spicy/Herbal, Sage, Woody,Uplifted, Happy, Rel..."
2,2,2,13 Dawgs,hybrid,"Apricot, Citrus, Grapefruit","Tingly, Creative, Hungry, Relaxed, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Apricot, Citrus, Grapefruit,Tingly, Creative, ..."
3,3,3,24K Gold,hybrid,"Citrus, Earthy, Orange","Happy, Relaxed, Euphoric, Uplifted, Talkative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...","Citrus, Earthy, Orange,Happy, Relaxed, Euphori..."
4,4,4,3 Bears Og,indica,"Sweet, Pungent, Earthy","Relaxed, Happy, Sleepy, Creative, Euphoric","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,3 Bears OG by Mephisto Genetics is an autoflow...,"Sweet, Pungent, Earthy,Relaxed, Happy, Sleepy,..."
5,5,5,3 Kings,hybrid,"Earthy, Sweet, Pungent","Relaxed, Euphoric, Happy, Uplifted, Hungry","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,"The 3 Kings marijuana strain, a holy trinity o...","Earthy, Sweet, Pungent,Relaxed, Euphoric, Happ..."
6,6,6,303 Og,indica,"Citrus, Pungent, Earthy","Relaxed, Happy, Euphoric, Uplifted, Giggly","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.2,The indica-dominant 303 OG is a Colorado strai...,"Citrus, Pungent, Earthy,Relaxed, Happy, Euphor..."
7,7,7,3D Cbd,sativa,"Earthy, Woody, Flowery","Uplifted, Focused, Happy, Talkative, Relaxed","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.6,3D CBD from Snoop Dogg’s branded line of canna...,"Earthy, Woody, Flowery,Uplifted, Focused, Happ..."
8,8,8,3Rd Coast Panama Chunk,hybrid,"Pine, Diesel","Energetic, Happy, Talkative, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.7,3rd Coast Panama Chunk by Bedford Grow is a cr...,"Pine, Diesel,Energetic, Happy, Talkative, Upli..."
9,9,9,3X Crazy,indica,"Earthy, Grape, Sweet","Relaxed, Tingly, Happy, Euphoric, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,"Also known as Optimus Prime, the indica-domina...","Earthy, Grape, Sweet,Relaxed, Tingly, Happy, E..."


In [22]:
tfidf = TfidfVectorizer(stop_words = 'english',
                      # ngram_range=(1,2),
                      # max_df=.5,
                      # min_df=3,
                       tokenizer= tokenize)

dtm = tfidf.fit_transform(data['everything'])

dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
dtm.head()

,ammonia,anxious,appetite,apple,apricot,arouse,aroused,berry,"berry,,dizzy",blue,...,tar,tea,tingly,tobacco,tree,tropical,uplifted,vanilla,violet,woody
0,0.0,0.161836,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.448400,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.115481,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.156250,0.0,0.0,0.350146
2,0.0,0.103582,0.0,0.0,0.614133,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.286995,0.0,0.0,0.0,0.140150,0.0,0.0,0.000000
3,0.0,0.143473,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.194124,0.0,0.0,0.000000
4,0.0,0.176918,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [36]:
# Fit on DTM
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [37]:
data.iloc[552]

Unnamed: 0                                                   552
Id                                                           552
Strain                                                    Citrix
Race                                                      hybrid
Flavors                        Citrus, Grapefruit, Orange, Lemon
Positive       Euphoric, Happy, Energetic, Uplifted, Aroused,...
Negative                            Dry Mouth, Paranoid, Anxious
Medical        Depression, Stress, Lack of Appetite, Nausea, ...
Rating                                                       4.3
Description    Citrix is a hybrid cannabis strain that draws ...
everything     Citrus, Grapefruit, Orange, Lemon,Euphoric, Ha...
Name: 552, dtype: object

In [38]:
nn.kneighbors([dtm.iloc[100]])  # The second set of values are the suggested strains.

(array([[0.        , 0.55213002, 0.66428793, 0.67274962, 0.67450085]]),
 array([[ 100, 1677, 1752,  506, 1412]]))

In [39]:
test_phrase = ["""
I want something earthy Euphoric dizzy Depression
"""]

In [40]:
new = tfidf.transform(test_phrase)

In [41]:
nn.kneighbors(new.todense())  # The second set of values are the suggested strains.

(array([[0.92677693, 0.98572156, 0.99225521, 1.00824068, 1.00956997]]),
 array([[ 530,  138, 2271,  167, 1857]]))

In [42]:
from sklearn.externals import joblib 
joblib.dump(tfidf, 'baseline_tfidf.pkl')

['baseline_tfidf.pkl']

In [43]:
joblib.dump(nn, 'baseline_model.pkl')

['baseline_model.pkl']

In [44]:
data.head()

,Unnamed: 0,Id,Strain,Race,Flavors,Positive,Negative,Medical,Rating,Description,everything
0,0,0,100 Og,hybrid,"Earthy, Sweet, Citrus","Creative, Energetic, Tingly, Euphoric, Relaxed","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.0,$100 OG is a 50/50 hybrid strain that packs a ...,"Earthy, Sweet, Citrus,Creative, Energetic, Tin..."
1,1,1,1024,sativa,"Spicy/Herbal, Sage, Woody","Uplifted, Happy, Relaxed, Energetic, Creative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,1024 is a sativa-dominant hybrid bred in Spain...,"Spicy/Herbal, Sage, Woody,Uplifted, Happy, Rel..."
2,2,2,13 Dawgs,hybrid,"Apricot, Citrus, Grapefruit","Tingly, Creative, Hungry, Relaxed, Uplifted","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Apricot, Citrus, Grapefruit,Tingly, Creative, ..."
3,3,3,24K Gold,hybrid,"Citrus, Earthy, Orange","Happy, Relaxed, Euphoric, Uplifted, Talkative","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.6,"Also known as Kosher Tangie, 24k Gold is a 60%...","Citrus, Earthy, Orange,Happy, Relaxed, Euphori..."
4,4,4,3 Bears Og,indica,"Sweet, Pungent, Earthy","Relaxed, Happy, Sleepy, Creative, Euphoric","Dizzy, Dry Mouth, Paranoid, Anxious","Depression, Stress, Fatigue",4.4,3 Bears OG by Mephisto Genetics is an autoflow...,"Sweet, Pungent, Earthy,Relaxed, Happy, Sleepy,..."


In [45]:
class strainSuggester():
    """
    Generates 5 suggested strains based on input text
    """
    def __init__(self):
        self.tfidf = joblib.load('baseline_tfidf.pkl')
        self.test_model = joblib.load('baseline_model.pkl')
    
    def suggestStrain(self, input, neighbors=5):
        vectorized = (tfidf.transform(input)).todense()
        results = nn.kneighbors(vectorized, neighbors)
        return results[0]

In [25]:
ss = strainSuggester()
ss.suggestStrain(test_phrase)

array([[1.04117704, 1.04117704, 1.04117704, 1.04117704, 1.04117704]])